In [1]:
import os
import numpy

In [2]:
file = os.path.join("atomistic","ucer2_hairpin_charmm","ucer2.itp")

In [3]:
outfile = open(file,'r')
data = outfile.readlines()
outfile.close()

In [4]:
line_counter,startline,endline= 1,0,0
for line in data:
    if line == '[ bonds ]\n':
        endline = line_counter - 2
    if line == '; nr type resnr residue atom cgnr charge\n':
        startline = line_counter
    line_counter += 1
dictdata = data[startline:endline]

In [5]:
master_dict = {}
masses = {}
charges = {}
for line in dictdata:
    line = line.split(" ")
    master_dict[line[0]] = line[1]
    charges[line[1]] = line[-1]

In [6]:
line_counter,startline,endline= 1,0,0
for line in data:
    if line == '[ bonds ]\n':
        startline = line_counter + 1
    if line == '[ angles ]\n':
        endline = line_counter - 2
    line_counter += 1
bondlist = data[startline:endline]

In [7]:
bond_params = []
translated_bonds = []
for bond in bondlist:
    bond = bond.split(' ')
    bond[0] = master_dict[str(bond[0])]
    bond[1] = master_dict[str(bond[1])]
    s = ' '
    bond = s.join(bond)
    translated_bonds.append(bond)
    if bond not in bond_params:
        bond_params.append(bond)


In [8]:
line_counter,startline,endline= 1,0,0
for line in data:
    if line == '[ angles ]\n':
        startline = line_counter + 1
    if line == '[ dihedrals ]\n':
        endline = line_counter - 2
    line_counter += 1
anglelist = data[startline:endline]
angle_params = []
translated_angles = []
for angle in anglelist:
    angle = angle.split(' ')
    angle[0] = master_dict[str(angle[0])]
    angle[1] = master_dict[str(angle[1])]
    angle[2] = master_dict[str(angle[2])]
    s = ' '
    angle = s.join(angle)
    translated_angles.append(angle)
    if angle not in angle_params:
        angle_params.append(angle)

In [9]:
line_counter,startline,endline= 1,0,0
for line in data:
    if line == '[ dihedrals ]\n':
        startline = line_counter + 1
    if line == '[ dihedrals ] ; impropers\n':
        endline = line_counter - 2
    line_counter += 1
dihedrallist = data[startline:endline]
dihedral_params = []
translated_dihedrals = []
for dihedral in dihedrallist:
    dihedral = dihedral.split(' ')
    dihedral[0] = master_dict[str(dihedral[0])]
    dihedral[1] = master_dict[str(dihedral[1])]
    dihedral[2] = master_dict[str(dihedral[2])]
    dihedral[3] = master_dict[str(dihedral[3])]
    s = ' '
    dihedral = s.join(dihedral)
    translated_dihedrals.append(dihedral)
    if dihedral not in dihedral_params:
        dihedral_params.append(dihedral)

In [10]:
line_counter,startline,endline= 1,0,0
for line in data:
    if line == '[ dihedrals ] ; impropers\n':
        startline = line_counter + 1
    if line == '[ pairs ]\n':
        endline = line_counter - 2
    line_counter += 1
improperlist = data[startline:endline]
improper_params = []
translated_impropers = []
for improper in improperlist:
    improper = improper.split(' ')
    improper[0] = master_dict[str(improper[0])]
    improper[1] = master_dict[str(improper[1])]
    improper[2] = master_dict[str(improper[2])]
    improper[3] = master_dict[str(improper[3])]
    s = ' '
    improper = s.join(improper)
    translated_impropers.append(improper)
    if improper not in improper_params:
        improper_params.append(improper)

In [11]:
atoms = []
for atom in master_dict.values():
    if atom not in atoms:
        atoms.append(atom)

In [12]:
output = open('translated.txt','w+')
output.write('<atomtypes>\n')
for atom in atoms:
    output.write(F'{atom}\n')
output.write('<atomtypes>\n')
output.write('\n')
output.write('<bonds>\n')
for bond in bond_params:
    output.write(F'{bond}')
output.write('<bonds>\n')
output.write('\n')
output.write('<angles>\n')
for angle in angle_params:
    output.write(F'{angle}')
output.write('<angles>\n')
output.write('\n')
output.write('<dihedrals>\n')
for dihedral in dihedral_params:
    output.write(F'{dihedral}')
output.write('<dihedrals>\n')
output.write('\n')
output.write('<impropers>\n')
for improper in improper_params:
    output.write(F'{improper}')
output.write('<impropers>\n')
output.close()

In [13]:
# convert things into correct format to add to master list
xml_bonds = []
for bond in bond_params:
    bond = bond.strip('\n')
    elements = bond.split(' ')
    xml_bond = F'    <Bond type1="{elements[0]}" type2="{elements[1]}" length="{elements[3]}" k="{elements[-1]}"/>\n'
    xml_bonds.append(xml_bond)
master_data = open('foyer_charmm.xml','r').readlines()
line_counter,startline,endline= 1,0,0
for line in master_data:
    if line == '  <HarmonicBondForce>\n':
        startline = line_counter
    if line == '  </HarmonicBondForce>\n':
        endline = line_counter - 1
    line_counter += 1
existing_bonds = master_data[startline:endline]
existing_bondtypes = []
for bond in existing_bonds:
    bond = bond.split('"')
    bondtype = [bond[1], bond[3]]
    alt = [bond[3], bond[1]]
    existing_bondtypes.append(bondtype)
    existing_bondtypes.append(alt)
out = open('bond_add.txt','w+')
for bond in xml_bonds:
    bond1 = bond.split('"')
    bondtype = [bond1[1], bond1[3]]
    if bondtype not in existing_bondtypes:
        out.write(bond)
out.close()


In [14]:
xml_angles = []
for angle in angle_params:
    angle = angle.strip('\n')
    elements = angle.split(' ')
    xml_angle = F'    <Angle type1="{elements[0]}" type2="{elements[1]}" type3="{elements[2]}" angle="{elements[4]}" k="{elements[-1]}"/>\n'
    xml_angles.append(xml_angle)
master_data = open('foyer_charmm.xml','r').readlines()
line_counter,startline,endline= 1,0,0
for line in master_data:
    if line == '  <HarmonicAngleForce>\n':
        startline = line_counter
    if line == '  </HarmonicAngleForce>\n':
        endline = line_counter - 1
    line_counter += 1
existing_angles = master_data[startline:endline]
existing_angletypes = []
for angle in existing_angles:
    angle = angle.split('"')
    existing_angletypes.append([angle[1], angle[3], angle[5]])
    existing_angletypes.append([angle[3], angle[1], angle[5]])
    existing_angletypes.append([angle[5], angle[3], angle[1]])
    existing_angletypes.append([angle[3], angle[5], angle[1]])
    existing_angletypes.append([angle[5], angle[1], angle[3]])
    existing_angletypes.append([angle[1], angle[5], angle[3]])
    
out = open('angle_add.txt','w+')
for angle in xml_angles:
    angle1 = angle.split('"')
    type = [angle1[1], angle1[3], angle1[5]]
    if type not in existing_angletypes:
        out.write(angle)
out.close()

In [15]:
def permutation(lst): 
  
    # If lst is empty then there are no permutations 
    if len(lst) == 0: 
        return [] 
  
    # If there is only one element in lst then, only 
    # one permuatation is possible 
    if len(lst) == 1: 
        return [lst] 
  
    # Find the permutations for lst if there are 
    # more than 1 characters 
  
    l = [] # empty list that will store current permutation 
  
    # Iterate the input(lst) and calculate the permutation 
    for i in range(len(lst)): 
       m = lst[i] 
  
       # Extract lst[i] or m from the list.  remLst is 
       # remaining list 
       remLst = lst[:i] + lst[i+1:] 
  
       # Generating all permutations where m is first 
       # element 
       for p in permutation(remLst): 
           l.append([m] + p) 
    return l 
  
xml_di = []
for di in dihedral_params:
    elements = di.split(' ')
    period = elements[-1].strip()
    rad = str(numpy.radians(float(elements[5])))
    k1 = elements[6]
    xml_d = F'    <Proper type1="{elements[0]}" type2="{elements[1]}" type3="{elements[2]}" type4="{elements[3]" periodicity1="{period}" phase1="{rad}" k1="{k1}"/>'
    xml_di.append(xml_d)
master_data = open('foyer_charmm.xml','r').readlines()
line_counter,startline,endline= 1,0,0
for line in master_data:
    if line == '  <PeriodicTorsionForce>\n':
        startline = line_counter
    if line == '  </PeriodicTorsionForce>\n':
        endline = line_counter - 1
    line_counter += 1
existing_di = master_data[startline:endline]
existing_ditypes = []
for di in existing_di:
    di = di.split('"')
    for existing in permutation([di[1], di[3], di[5], di[7]]):
        existing_ditypes.append(existing)   
    
out = open('proper_add.txt','w+')
for di in xml_di:
    di1 = di.split('"')
    type = [di1[1], di1[3], di1[5], di1[7]]
    if type not in existing_ditypes:
        out.write(di)
out.close()

SyntaxError: f-string: unterminated string (<ipython-input-15-9ac883769253>, line 37)